### 데이터 받아오기

In [1]:
import json
import numpy as np
import random
import nltk

In [2]:
with open("Books_5.json","r") as data_file:
    raw_data=data_file.readlines()

In [3]:
json_list = []
for i in range(len(raw_data)):
    try:
        json_data = json.loads(raw_data[i])
        json_list.append(json_data)
    except:
        print(raw_data[i],i)

### 데이터 분리하기

In [4]:
review_data = []

for i in range(len(json_list)):
    review_dict={}
    review_dict["reviewText"] = json_list[i]["reviewText"]
    review_dict["overall"] = json_list[i]["overall"]
    review_data.append(review_dict)

In [5]:
reviewText = []
for i in range(len(review_data)):
    reviewText.append(review_data[i]["reviewText"])

In [6]:
reviewText[9]

"An amazing work. Realizing extensive use of Biblical imagery and sentence structure, &#34;The Prophet&#34; by Khalil Gibran is a literary classic. Influencing the Free Love movement of the 1960's, Gibran's master work explores themes of love, longing and loss."

In [7]:
y_data = []
for i in range(len(review_data)):
    y_data.append(review_data[i]["overall"])

In [8]:
from collections import Counter
Counter(y_data)

Counter({1.0: 323833, 2.0: 415110, 3.0: 955189, 4.0: 2223094, 5.0: 4980815})

<h3>데이터 전처리</h3>

In [9]:
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import re

In [10]:
def review_to_words(raw_review):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    
   
    cleaned_text_list=[]
    for i in range(len(raw_review)):
        
        review_text = BeautifulSoup(raw_review[i]).get_text()
        letters_only = re.sub("[^a-zA-Z]", " ", review_text)
        words = letters_only.lower().split()
        cleaned_text = " ".join(words)
        cleaned_text_list.append(cleaned_text)

    return cleaned_text_list

       

In [11]:
pre_data=review_to_words(reviewText)

c:\users\jangy\miniconda3\envs\bi_project\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 170 of the file c:\users\jangy\miniconda3\envs\bi_project\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))
c:\users\jangy\miniconda3\envs\bi_project\lib\site-packages\bs4\__init__.py:282: UserWarning: "http://www.examiner.com/review/maelstroms-the-bangkok-of-timothy-halliman?cid=db_articles" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed

In [12]:
len(pre_data)

8898041

### 데이터 셔플

In [13]:
c = list(zip(pre_data, y_data))
random.shuffle(c)

In [14]:
a, b = zip(*c)

In [15]:
pre_text=list(a)
overall=np.array(list(b))

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer='word', sublinear_tf=True,lowercase=True)
vectorizer = TfidfVectorizer(ngram_range=(1,2))

tfidf = vectorizer.fit_transform(pre_text)

### 가장 작은 평점으로 갯수 맞추기

In [17]:
from collections import Counter
length=sorted(Counter(overall).most_common())[0][1]

In [18]:
search=[1.0,2.0,3.0,4.0,5.0]
count=0
text=[]
y=[]

for i in search:
    count=0
    for index,value in enumerate(overall):
        if (count==length): break
        elif (i==value):
            text.append(pre_text[index])
            y.append(value)
            count=count+1

In [19]:
Counter(y)

Counter({1.0: 323833, 2.0: 323833, 3.0: 323833, 4.0: 323833, 5.0: 323833})

In [20]:
print(len(text), len(y))

1619165 1619165


<h3>데이터 셔플 다시</h3>

In [21]:
c = list(zip(text, y))
random.shuffle(c)

In [22]:
a, b = zip(*c)

In [23]:
text_data=list(a)
overall_data=np.array(list(b))

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer='word', sublinear_tf=True,lowercase=True)
vectorizer = TfidfVectorizer(ngram_range=(1,2))

tfidf = vectorizer.fit_transform(text_data)

In [25]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(tfidf , overall_data , test_size=0.33, random_state=100)

In [26]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((1084840, 20596668), (1084840,), (534325, 20596668), (534325,))

### 예측

#### Multiclass SVM

In [27]:
# from sklearn import datasets
# from sklearn.multiclass import OneVsRestClassifier
# from sklearn.svm import LinearSVC

# result = OneVsRestClassifier(LinearSVC(random_state=100)).fit(x_train, y_train)
# result = result.predict(x_test)

In [28]:
# from sklearn.metrics import accuracy_score
# accuracy_score(result ,y_test)

0.55077153417863656

In [29]:
from sklearn.metrics import classification_report
print(classification_report(y_test, result))

             precision    recall  f1-score   support

        1.0       0.63      0.69      0.66    106659
        2.0       0.48      0.44      0.46    107247
        3.0       0.50      0.46      0.48    106991
        4.0       0.49      0.47      0.48    106970
        5.0       0.63      0.70      0.66    106458

avg / total       0.54      0.55      0.55    534325



### LogisticRegression

In [30]:
from sklearn import linear_model
logreg = linear_model.LogisticRegression()

logreg.fit(x_train, y_train)
logreg.predict(x_test)==y_test

array([ True,  True, False, ...,  True, False, False], dtype=bool)

In [31]:
from sklearn.metrics import accuracy_score
accuracy_score(logreg.predict(x_test), y_test)

0.58050250315819019

In [32]:
from sklearn.metrics import precision_score
precision_score(logreg.predict(x_test), y_test,average=None) 

array([ 0.71789535,  0.46698742,  0.49167687,  0.49688698,  0.73049466])

In [33]:
from sklearn import metrics
fpr_1, tpr_1, thresholds = metrics.roc_curve(y_test, logreg.predict(x_test), pos_label=1)
metrics.auc(fpr_1, tpr_1)

0.11702783317605014

In [34]:
fpr_2, tpr_2, thresholds = metrics.roc_curve(y_test, logreg.predict(x_test), pos_label=2)
metrics.auc(fpr_2, tpr_2)

0.2684977087352195

In [44]:
fpr_3, tpr_3, thresholds = metrics.roc_curve(y_test, logreg.predict(x_test), pos_label=3) ######여기다시
metrics.auc(fpr_3, tpr_3)

0.49178580122519733

In [36]:
fpr_4, tpr_4, thresholds = metrics.roc_curve(y_test, logreg.predict(x_test), pos_label=4)
metrics.auc(fpr_4, tpr_4)

0.73553431905837741

In [37]:
fpr_5, tpr_5, thresholds = metrics.roc_curve(y_test, logreg.predict(x_test), pos_label=5)
metrics.auc(fpr_5, tpr_5)

0.88816411987429345

In [38]:
print(y_test[:20], logreg.predict(x_test)[:20])

[ 1.  1.  4.  1.  1.  2.  4.  3.  1.  2.  3.  5.  1.  2.  4.  2.  4.  1.
  1.  4.] [ 1.  1.  5.  1.  1.  2.  4.  3.  1.  2.  3.  5.  2.  2.  5.  1.  5.  1.
  2.  4.]


In [39]:
logreg.predict(x_test)[:10]

array([ 1.,  1.,  5.,  1.,  1.,  2.,  4.,  3.,  1.,  2.])

In [40]:
result=logreg.predict(x_test)

In [41]:
from sklearn.metrics import classification_report
print(classification_report(y_test, result))

             precision    recall  f1-score   support

        1.0       0.64      0.72      0.68    106659
        2.0       0.52      0.47      0.49    107247
        3.0       0.54      0.49      0.51    106991
        4.0       0.53      0.50      0.51    106970
        5.0       0.65      0.73      0.69    106458

avg / total       0.57      0.58      0.58    534325



### INPUT 

In [42]:
texts = ["""I enjoyed this book, as long as I kept in mind that the book was written for young adults.  
         The characters were believable and it was easy to identify with their pain.""",
        "I think the author gets lost in his own thoughts and trys to make storys of something that's not there.",
        """I was very excited to start reading this book and although I did like it, 
        I thought that a lot of the words to describe body parts was kind of high-schoolish.  
        Didn't care for the cliff hanging ending.""",
        """I don't know if you are familiar with FANFIC on the internet but that what this book reminded me of.  
        I love the series and thought I would like the books...not so much.""",
        """As I love western era romances, this one was wonderful.  An easy read and ended the way we always hope....happily!"""]
overall=[4,1,3,2,5]
vecs = vectorizer.transform(texts) 

In [43]:
print(overall,logreg.predict(vecs))

[4, 1, 3, 2, 5] [ 4.  1.  3.  3.  5.]
